In [1]:
import numpy as np
import pandas as pd
#for reading in data properly
import ast
import json

import gensim
from gensim.models import Doc2Vec
from gensim.models import Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.model_selection import train_test_split
from sklearn import utils

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

In [2]:
all_data = pd.read_csv('train.csv')
all_data = all_data.dropna(subset=['overview', 'genres']) #drop cols without overview or genre (data we use or labels)

In [3]:
def text_to_list(x):
    if pd.isna(x):
        return ''
    else:
        return ast.literal_eval(x)

def parse_json(x):
    try:
        return json.loads(x.replace("'", '"'))[0]['name']
    except:
        return ''
    
def parse_genres_json(x):
    try:
        json_genres = json.loads(x.replace("'", '"'))
        numElems = len(json_genres)
        ret = [0]*len(genre_dict) #number of genres we are looking at
        for i in range(numElems):
            genre_str = (json_genres[i]['name'])
            if genre_str in genre_map.keys():
                ret[genre_dict[genre_map[genre_str]]] = 1
        return ret
    except Exception as excep:
        print('Exception' + str(excep))
        return ''

In [4]:
genre_dict = {}
genre_dict['Action-Adventure'] = 0
genre_dict['Romance'] = 1
genre_dict['Horror-Thriller'] = 2
genre_dict['Comedy'] = 3
genre_dict['Science Fiction'] = 4
#genre_dict['Drama'] = 5
genre_dict

{'Action-Adventure': 0,
 'Romance': 1,
 'Horror-Thriller': 2,
 'Comedy': 3,
 'Science Fiction': 4}

In [5]:
genre_map = {}
genre_map['Adventure'] = 'Action-Adventure'
genre_map['Romance'] = 'Romance'
genre_map['Horror'] = 'Horror-Thriller'
genre_map['Thriller'] = 'Horror-Thriller'
genre_map['Comedy'] = 'Comedy'
#genre_map['War'] = 'Action-Adventure'#not sure about this
genre_map['Action'] = 'Action-Adventure'
genre_map['Science Fiction'] = 'Science Fiction'
#genre_map['Drama'] = 'Drama'

In [6]:
def getGenresVects():
    y = all_data['genres']
    ret = y.apply(parse_genres_json)
    all_data['genres_vect'] = ret
    return ret

In [7]:
labels_vects = getGenresVects() #get label vectors for genres indexed by indexes in genre_dict

In [8]:
#put to lower case, remove punctation
def cleanText(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    text = ' '.join(no_stopword_text)
    text = re.sub(r'[^a-z A-Z0-9]', "", text) #maybe shouldn't remove punction between words here?
    text = text.lower()
    return text
all_data['cleanOverview'] = all_data['overview'].apply(cleanText)

In [9]:
all_data = all_data[all_data.genres_vect.map(sum) > 0]

In [10]:
#logistic regression data
lr_data = all_data[['cleanOverview', 'genres_vect', 'overview']]

In [11]:
train, test = train_test_split(lr_data, test_size=0.2, random_state=42)

CNN STUFF here

In [12]:
#get word embeddings
x = train['cleanOverview'].values.tolist()
y = train['genres_vect']

In [13]:
x_test = test['cleanOverview'].values.tolist()
y_test = test['genres_vect']

In [14]:
y_train = y.tolist()
y_train = np.array(y_train)

In [15]:
y_test = y_test.tolist()
y_test = np.array(y_test)

In [16]:
tok = [word_tokenize(sent) for sent in x]

In [17]:
word_vec_len = 32
model = Word2Vec(tok, min_count = 2, size=word_vec_len)

In [18]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words_kept = 100000 #using 100000 most popular words, use throughout

tokenizer = Tokenizer(num_words_kept)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

max_seq_len = 150

x_train_seq = pad_sequences(sequences, maxlen=max_seq_len)

Using TensorFlow backend.


In [19]:
test_sequences = tokenizer.texts_to_sequences(x_test)
x_test_seq = pad_sequences(test_sequences, maxlen=max_seq_len)

In [20]:
embeddings_index = {}
for w in model.wv.vocab.keys():
    embeddings_index[w] = model.wv[w]


embedding_matrix = np.zeros((num_words_kept, word_vec_len))
for word, i in tokenizer.word_index.items():
    if i >= num_words_kept:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [21]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def get_per_label_metrics(real_labels_matrix, predictions_labels_matrix):
    for genre in genre_dict.keys():
        index = genre_dict[genre]
        real_labels_vect = real_labels_matrix[:, index]
        prediction_vect = predictions_labels_matrix[:,index]
        print("Accuruacy for " + genre + ": " + str(accuracy_score(real_labels_vect, prediction_vect)))
        print("Precision for " + genre + ": " + str(precision_score(real_labels_vect, prediction_vect)))
        print("Recall for " + genre + ": " + str(recall_score(real_labels_vect, prediction_vect)))
        print()

In [22]:
#size of intersection of predicted and actual labels divided by size of their union for each datapoint tested on
#sum those and then divide by number of datapoints
#vectorized for speed
def multi_label_accuracy(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    union = real_labels_matrix | predictions_labels_matrix
    #sum(array.T) gets number of 1s in row
    row_wise_accuracy = sum(intersection.T) / sum(union.T)
    return sum(row_wise_accuracy) / real_labels_matrix.shape[0]

#size of intersection of predicted and actual labels divided by size of predicted set for each datapoint tested on
#sum those and divide by number of datapoints
#if no predicted labels, don't count that row towards the precision as that would be undefined
def multi_label_precision(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    precision_sum = 0
    num_rows = 0
    for row in range(intersection.shape[0]):
        if sum(predictions_labels_matrix[row]) > 0: #if there is at least one prediction for this row
            num_rows += 1
            precision_sum += sum(intersection[row]) / sum(predictions_labels_matrix[row])
    if num_rows == 0:
        return 0#no labels predicted at all will give us 0 precision as precision makes no sense here
    return precision_sum / num_rows

#size of intersection of predicted and actual labels divided by size of real label set for each datapoint tested on
#sum those and divide by number of datapoints
#all datapoints should have at least 1 real label in this data set
#vectorized for speed
def multi_label_recall(real_labels_matrix, predictions_labels_matrix):
    #binary so set intersection is and operator
    intersection = real_labels_matrix & predictions_labels_matrix
    #set union for binary is same as or operator
    #sum(array.T) gets number of 1s in row
    row_wise_recall = sum(intersection.T) / sum(real_labels_matrix.T)
    return sum(row_wise_recall) / real_labels_matrix.shape[0]

#lower is better
def hamming_loss(real_labels_matrix, predictions_labels_matrix):
    return (np.logical_xor(real_labels_matrix, predictions_labels_matrix)).sum()/(real_labels_matrix.shape[0] * real_labels_matrix.shape[1])

import keras.backend as K

#metric for keras for early stopping
#takes in raw labels from kerass (not yet converted to 0 and 1s)
#NOT the same as accuracy, this is total labels correctly identified divided by union of total labels
#this weights rows with more labels higher, where accruacy does not, but this is still a good metric for early stopping
def raw_multi_label_accuracy(y_true, y_pred):
    positives = K.greater_equal(y_pred, 0.5)
    positives = K.cast(positives, K.floatx())
    new_y_pred = positives #+ ((1-positives)*y_pred)
    intersection = y_true * new_y_pred
    union = 1 -((1-y_true)*(1-new_y_pred))
    accuracy = K.sum(intersection) / K.sum(union)
    return accuracy
    

In [23]:
from keras.callbacks import EarlyStopping
#for early stopping only after certain number of epochs. wait until delay epochs until early stopping
class DelayedEarlyStopping(EarlyStopping):
    def __init__(self, monitor, min_delta=0, patience=0, verbose=0, mode='auto', delay = 100):
        super(DelayedEarlyStopping, self).__init__()
        self.delay = delay

    def on_epoch_end(self, epoch, logs=None):
        if epoch > self.delay:
            super().on_epoch_end(epoch, logs)

In [ ]:
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.regularizers import l2

model_cnn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
#e = Embedding(num_words_kept, word_vec_len, input_length=max_seq_len, trainable=True)
model_cnn.add(e)
model_cnn.add(Conv1D(filters=50, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
model_cnn.add(Dropout(.5))
model_cnn.add(Dense(50, activation='relu', kernel_regularizer=l2(0.01)))
model_cnn.add(Dropout(.5))
model_cnn.add(Dense(len(genre_dict), activation='sigmoid'))
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
#model_cnn_01.fit(x_train_seq, y_train, validation_data=(x_val_seq, y_validation), epochs=5, batch_size=32, verbose=2)
model_cnn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=200)], epochs=1000, batch_size=100, verbose=2)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 4s - loss: 2.0877 - raw_multi_label_accuracy: 0.2180 - val_loss: 1.7679 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 1s - loss: 1.5937 - raw_multi_label_accuracy: 0.1558 - val_loss: 1.3683 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 1s - loss: 1.2566 - raw_multi_label_accuracy: 0.1077 - val_loss: 1.0980 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 1s - loss: 1.0259 - raw_multi_label_accuracy: 0.0908 - val_loss: 0.9151 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 1s - loss: 0.8760 - raw_multi_label_accuracy: 0.0884 - val_loss: 0.7966 - val_raw_multi_label_accura

Epoch 55/1000
 - 1s - loss: 0.3310 - raw_multi_label_accuracy: 0.6441 - val_loss: 1.0216 - val_raw_multi_label_accuracy: 0.2476
Epoch 56/1000
 - 1s - loss: 0.3258 - raw_multi_label_accuracy: 0.6547 - val_loss: 1.0006 - val_raw_multi_label_accuracy: 0.2557
Epoch 57/1000
 - 1s - loss: 0.3223 - raw_multi_label_accuracy: 0.6561 - val_loss: 1.0983 - val_raw_multi_label_accuracy: 0.2544
Epoch 58/1000
 - 1s - loss: 0.3178 - raw_multi_label_accuracy: 0.6558 - val_loss: 1.0515 - val_raw_multi_label_accuracy: 0.2422
Epoch 59/1000
 - 1s - loss: 0.3064 - raw_multi_label_accuracy: 0.6880 - val_loss: 1.1016 - val_raw_multi_label_accuracy: 0.2517
Epoch 60/1000
 - 1s - loss: 0.3062 - raw_multi_label_accuracy: 0.6852 - val_loss: 1.0676 - val_raw_multi_label_accuracy: 0.2621
Epoch 61/1000
 - 1s - loss: 0.3043 - raw_multi_label_accuracy: 0.6764 - val_loss: 1.1220 - val_raw_multi_label_accuracy: 0.2718
Epoch 62/1000
 - 1s - loss: 0.3046 - raw_multi_label_accuracy: 0.6786 - val_loss: 1.1411 - val_raw_multi

Epoch 119/1000
 - 1s - loss: 0.2207 - raw_multi_label_accuracy: 0.7847 - val_loss: 1.3623 - val_raw_multi_label_accuracy: 0.3009
Epoch 120/1000
 - 1s - loss: 0.2170 - raw_multi_label_accuracy: 0.7966 - val_loss: 1.3945 - val_raw_multi_label_accuracy: 0.2968
Epoch 121/1000
 - 1s - loss: 0.2198 - raw_multi_label_accuracy: 0.7847 - val_loss: 1.4212 - val_raw_multi_label_accuracy: 0.3142
Epoch 122/1000
 - 1s - loss: 0.2225 - raw_multi_label_accuracy: 0.7816 - val_loss: 1.3738 - val_raw_multi_label_accuracy: 0.3095
Epoch 123/1000
 - 1s - loss: 0.2125 - raw_multi_label_accuracy: 0.7998 - val_loss: 1.3630 - val_raw_multi_label_accuracy: 0.3033
Epoch 124/1000
 - 1s - loss: 0.2146 - raw_multi_label_accuracy: 0.7959 - val_loss: 1.3978 - val_raw_multi_label_accuracy: 0.2951
Epoch 125/1000
 - 1s - loss: 0.2199 - raw_multi_label_accuracy: 0.7858 - val_loss: 1.4430 - val_raw_multi_label_accuracy: 0.2995
Epoch 126/1000
 - 1s - loss: 0.2204 - raw_multi_label_accuracy: 0.7881 - val_loss: 1.4169 - val_r

Epoch 183/1000
 - 1s - loss: 0.1908 - raw_multi_label_accuracy: 0.8166 - val_loss: 1.5881 - val_raw_multi_label_accuracy: 0.3208
Epoch 184/1000
 - 1s - loss: 0.1860 - raw_multi_label_accuracy: 0.8269 - val_loss: 1.5786 - val_raw_multi_label_accuracy: 0.3211
Epoch 185/1000
 - 1s - loss: 0.1912 - raw_multi_label_accuracy: 0.8123 - val_loss: 1.6274 - val_raw_multi_label_accuracy: 0.3228
Epoch 186/1000
 - 1s - loss: 0.1853 - raw_multi_label_accuracy: 0.8208 - val_loss: 1.5991 - val_raw_multi_label_accuracy: 0.3195
Epoch 187/1000
 - 1s - loss: 0.1849 - raw_multi_label_accuracy: 0.8239 - val_loss: 1.6297 - val_raw_multi_label_accuracy: 0.3169
Epoch 188/1000
 - 1s - loss: 0.1836 - raw_multi_label_accuracy: 0.8278 - val_loss: 1.6911 - val_raw_multi_label_accuracy: 0.3090
Epoch 189/1000
 - 1s - loss: 0.1829 - raw_multi_label_accuracy: 0.8289 - val_loss: 1.6059 - val_raw_multi_label_accuracy: 0.3178
Epoch 190/1000
 - 1s - loss: 0.1858 - raw_multi_label_accuracy: 0.8197 - val_loss: 1.5792 - val_r

In [ ]:
def nn_output_to_predictions(res):
    label_predictions = []
    for i in range(res.shape[0]):
        pred = [0]*len(genre_dict)
        for j in range(res.shape[1]):
            if res[i][j] >= .5:
                pred[j] = 1
        label_predictions.append(pred)
    return np.array(label_predictions)

In [ ]:
predictions = nn_output_to_predictions(model_cnn.predict(x_test_seq))

In [ ]:
y_test[:,0].sum()

187

In [ ]:
predictions[:,0].sum()

246

In [ ]:
predictions[0]

array([0, 0, 0, 1, 0])

In [ ]:
multi_label_accuracy(y_test, predictions)

0.36687370600414065

In [ ]:
multi_label_precision(y_test, predictions)

0.4689440993788821

In [ ]:
multi_label_recall(y_test, predictions)

0.46670117322291227

In [ ]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 66.50103519668737


In [ ]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.5424430641821946
Precision for Action-Adventure: 0.43089430894308944
Recall for Action-Adventure: 0.5668449197860963

Accuruacy for Romance: 0.7370600414078675
Precision for Romance: 0.29411764705882354
Recall for Romance: 0.14150943396226415

Accuruacy for Horror-Thriller: 0.5734989648033126
Precision for Horror-Thriller: 0.46534653465346537
Recall for Horror-Thriller: 0.4895833333333333

Accuruacy for Comedy: 0.5921325051759835
Precision for Comedy: 0.536697247706422
Recall for Comedy: 0.5492957746478874

Accuruacy for Science Fiction: 0.8799171842650103
Precision for Science Fiction: 0.4
Recall for Science Fiction: 0.07142857142857142



CNN but with multiple filter sizes so we don't just filter on group of words at a time

In [ ]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

model_input = Input(shape=(max_seq_len,), dtype='int32')

e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)(model_input)
two_word_filter = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(e)
two_word_filter = GlobalMaxPooling1D()(two_word_filter)
three_word_filter = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(e)
three_word_filter = GlobalMaxPooling1D()(three_word_filter)
four_word_filter = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(e)
four_word_filter = GlobalMaxPooling1D()(four_word_filter)
merged = concatenate([two_word_filter, three_word_filter, four_word_filter], axis=1)

merged = Dense(256, activation='relu', kernel_regularizer=l2(0.01))(merged)
merged = Dropout(0.5)(merged)
merged = Dense(len(genre_dict))(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[model_input], outputs=[output])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])

In [ ]:
model.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000
 - 4s - loss: 2.8865 - raw_multi_label_accuracy: 0.0867 - val_loss: 2.2903 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 2/1000
 - 3s - loss: 1.9246 - raw_multi_label_accuracy: 0.0448 - val_loss: 1.5395 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 3/1000
 - 4s - loss: 1.3227 - raw_multi_label_accuracy: 0.0285 - val_loss: 1.0945 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 4/1000
 - 4s - loss: 0.9769 - raw_multi_label_accuracy: 0.0267 - val_loss: 0.8523 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 5/1000
 - 6s - loss: 0.7883 - raw_multi_label_accuracy: 0.0196 - val_loss: 0.7164 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 6/1000
 - 4s - loss: 0.6907 - raw_multi_label_accuracy: 0.0198 - val_loss: 0.6514 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 7/1000
 - 3s - loss: 0.6352 - raw_multi_label_accuracy: 0.0231 - val_loss: 0.6189 - val_raw_multi_label_accuracy: 0.0000e+00
Epoch 8/1000
 - 3s - loss: 0.6003 - 

In [ ]:
predictions = nn_output_to_predictions(model.predict(x_test_seq))

In [ ]:
multi_label_accuracy(y_test, predictions)

0.39233954451345754

In [ ]:
multi_label_precision(y_test, predictions)

0.6348039215686274

In [ ]:
multi_label_recall(y_test, predictions)

0.4342650103519669

In [ ]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

Percent of correctly decided label decisions: 73.58178053830228


In [ ]:
get_per_label_metrics(y_test, predictions)

Accuruacy for Action-Adventure: 0.6459627329192547
Precision for Action-Adventure: 0.5645161290322581
Recall for Action-Adventure: 0.37433155080213903

Accuruacy for Romance: 0.7805383022774327
Precision for Romance: 0.5
Recall for Romance: 0.16037735849056603

Accuruacy for Horror-Thriller: 0.7163561076604554
Precision for Horror-Thriller: 0.7131782945736435
Recall for Horror-Thriller: 0.4791666666666667

Accuruacy for Comedy: 0.6542443064182195
Precision for Comedy: 0.6127450980392157
Recall for Comedy: 0.5868544600938967

Accuruacy for Science Fiction: 0.8819875776397516
Precision for Science Fiction: 0.4
Recall for Science Fiction: 0.03571428571428571



Regular Neural Network

In [ ]:
normal_nn = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
normal_nn.add(e)
normal_nn.add(Flatten())
normal_nn.add(Dense(256, activation='relu'))
normal_nn.add(Dense(len(genre_dict), activation='sigmoid'))
normal_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
normal_nn.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)

Train on 1738 samples, validate on 194 samples
Epoch 1/1000


In [ ]:
predictions = nn_output_to_predictions(normal_nn.predict(x_test_seq))

In [ ]:
multi_label_accuracy(y_test, predictions)

In [ ]:
multi_label_precision(y_test, predictions)

In [ ]:
multi_label_recall(y_test, predictions)

In [ ]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

In [ ]:
get_per_label_metrics(y_test, predictions)

LSTM

In [ ]:
from keras.layers import LSTM
lstm_model = Sequential()
e = Embedding(num_words_kept, word_vec_len, weights=[embedding_matrix], input_length=max_seq_len, trainable=True)
lstm_model.add(e)
lstm_model.add(LSTM(100, dropout=0.25, recurrent_dropout=0.25))
lstm_model.add(Dense(len(genre_dict), activation='sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[raw_multi_label_accuracy])
lstm_model.fit(x_train_seq, y_train, validation_split = .1, callbacks = [DelayedEarlyStopping(monitor = 'val_raw_multi_label_accuracy', patience = 5, delay=25)], epochs=1000, batch_size=100, verbose=2)

In [ ]:
predictions = nn_output_to_predictions(lstm_model.predict(x_test_seq))

In [ ]:
multi_label_accuracy(y_test, predictions)

In [ ]:
multi_label_precision(y_test, predictions)

In [ ]:
multi_label_recall(y_test, predictions)

In [ ]:
print("Percent of correctly decided label decisions: " + str(100* (1-hamming_loss(y_test, predictions))))

In [ ]:
get_per_label_metrics(y_test, predictions)